# CS450 Final

Katya Griffiths-Julien

Jerry Lau

In [ ]:
# import files
from google.colab import files
uploaded = files.upload()

Saving worldcitiespop.csv to worldcitiespop.csv


## Problem 2 Sparkifying world cities

In [ ]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=b65a80a41351ed03239144d3f05eb96528b43faf9864ed39c88cbd4af6a42304
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.5.0


In [ ]:
# check if file can be read, count total lines
import csv

with open('worldcitiespop.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip the header row
    print(sum(1 for row in reader))

3173958


In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext
import numpy as np


spark = SparkSession.builder.appName("MyName").getOrCreate()

filepath = 'worldcitiespop.csv'

df = spark.read.csv(filepath, header=True, inferSchema=True)

# Print the schema of the DataFrame
df.printSchema()

dfNew = df.filter(col(df.columns[4]).isNotNull())

print(dfNew.count())
#print(df.count())

root
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- AccentCity: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)

47980


In [ ]:
from pyspark.sql.functions import sum, min, max, avg

dfNew.select(sum(dfNew["population"])).show()
dfNew.select(min(dfNew["population"])).show()
dfNew.select(max(dfNew["population"])).show()
dfNew.select(avg(dfNew["population"])).show()


+---------------+
|sum(population)|
+---------------+
|     2289584999|
+---------------+

+---------------+
|min(population)|
+---------------+
|              7|
+---------------+

+---------------+
|max(population)|
+---------------+
|       31480498|
+---------------+

+-----------------+
|  avg(population)|
+-----------------+
|47719.57063359733|
+-----------------+



In [ ]:
# count per 10 power

from pyspark.sql.functions import log10, floor

# Round each integer down to its nearest power of 10
dfNew = dfNew.withColumn("rounded_value", floor(log10(dfNew['Population'])))

# Group the DataFrame by the rounded values and count the number of occurrences of each value
counts = dfNew.groupBy("rounded_value").count()

# Print the counts to the console
print(counts.orderBy("rounded_value").show())

+-------------+-----+
|rounded_value|count|
+-------------+-----+
|            0|    5|
|            1|  174|
|            2| 2187|
|            3|20537|
|            4|21550|
|            5| 3248|
|            6|  269|
|            7|   10|
+-------------+-----+

None


In [ ]:
from pyspark.sql.functions import desc

# Sort the dataframe by the "Population" column in descending order
sorted_df = dfNew.orderBy(desc("Population"))

# Select the top 10 cities with the largest population
top_10_cities = sorted_df.limit(10)

# Display the results
top_10_cities.show()



+-------+---------+----------+------+----------+----------+----------+-------------+
|Country|     City|AccentCity|Region|Population|  Latitude| Longitude|rounded_value|
+-------+---------+----------+------+----------+----------+----------+-------------+
|     jp|    tokyo|     Tokyo|    40|  31480498|    35.685|139.751389|            7|
|     cn| shanghai|  Shanghai|    23|  14608512| 31.045556|121.399722|            7|
|     in|   bombay|    Bombay|    16|  12692717|    18.975| 72.825833|            7|
|     pk|  karachi|   Karachi|    05|  11627378|   24.9056|   67.0822|            7|
|     in|    delhi|     Delhi|    07|  10928270| 28.666667| 77.216667|            7|
|     in|new delhi| New Delhi|    07|  10928270|      28.6|      77.2|            7|
|     ph|   manila|    Manila|    D9|  10443877|   14.6042|  120.9822|            7|
|     ru|   moscow|    Moscow|    48|  10381288| 55.752222| 37.615556|            7|
|     kr|    seoul|     Seoul|    11|  10323448|   37.5985|  126.

In [ ]:
from pyspark.sql.functions import sum, col

totalPop = dfNew.select(sum(dfNew["population"])).first()[0]

top10Pop = 0
for row in top_10_cities.collect():
  top10Pop += row["Population"]

print(top10Pop/totalPop)

0.05827942402587343


In [ ]:
from pyspark.sql.functions import sum, desc

import numpy as np

# Group the dataframe by the "region" column and calculate the sum of the "population" column
region_populations = dfNew.groupBy("region").agg(sum("population").alias("total_population"))

# Sort the dataframe by the "total_population" column in descending order
sorted_regions = region_populations.orderBy(desc("total_population"))

sorted_regions.show()

# Select the region with the highest population
highest_population_region = sorted_regions.select("region").first()[0]

# Display the results
print("The region with the highest population is {}.".format(highest_population_region))


+------+----------------+
|region|total_population|
+------+----------------+
|    04|       112249869|
|    07|        99634521|
|    02|        90271668|
|    05|        88272385|
|    16|        71422264|
|    08|        69386610|
|    09|        68742631|
|    11|        61477197|
|    01|        60187116|
|    06|        58191980|
|    15|        57366367|
|    10|        52912641|
|    13|        50762945|
|    19|        49067537|
|    23|        48056875|
|    30|        47565528|
|    25|        44541463|
|    12|        44192013|
|    40|        43122879|
|    27|        42540035|
+------+----------------+
only showing top 20 rows

The region with the highest population is 04.


In [ ]:
from pyspark.sql.functions import sum

# Group the dataframe by the "Country" column and calculate the sum of the "Population" column
country_populations = dfNew.groupBy("Country").agg(sum("Population").alias("total_population"))

# Filter the dataframe to include only the countries where the total population is greater than the specified value
specified_value = 10000000
filtered_countries = country_populations.filter(country_populations["total_population"] > specified_value)

# Display the results
print(filtered_countries.count())

for row in filtered_countries.collect():
    print(row["Country"])


41
eg
cl
jp
cn
bd
ir
in
au
gb
cd
co
de
br
es
kr
it
ar
ca
fr
id
us
ro
vn
pl
za
pk
tw
tr
pe
ph
nl
ve
my
ru
th
ng
ma
mm
ua
mx
tz


In [ ]:
from pyspark.sql.functions import avg, col

# Define the threshold population
threshold_population = 1000000

# Filter the dataframe to include only cities with a population above the threshold
filtered_cities = dfNew.filter(df["Population"] > threshold_population)

# Group the dataframe by the "Country" column and calculate the average of the "Population" column
average_populations = filtered_cities.groupBy("Country").agg(avg("Population").alias("average_population"))

# Display the results

print(average_populations.count())

for row in average_populations.collect():
    print("{}: {}".format(row["Country"], row["average_population"]))


89
eg: 4663201.333333333
ge: 1049516.0
cl: 4837248.0
jp: 6454776.666666667
ao: 2776125.0
by: 1742123.0
cn: 2486771.717948718
bd: 3917803.0
ir: 1498211.1666666667
ke: 2750561.0
kh: 1573523.0
in: 2921454.909090909
au: 2497827.8
ae: 1137376.0
iq: 5672516.0
be: 1019022.0
gb: 7421228.0
cd: 4581320.0
ec: 1675921.5
ci: 3692570.0
co: 3218984.25
de: 2126113.6666666665
br: 2513657.2666666666
es: 2336511.0
kr: 10323448.0
af: 3043589.0
it: 1900319.5
ar: 1329752.0
hu: 1708088.0
ca: 3239556.6666666665
cm: 1318795.0
gh: 1716128.5
do: 2202016.0
ht: 1234750.0
bg: 1062065.0
at: 1569315.0
dz: 1980242.0
cz: 1154508.0
kz: 1204820.0
az: 1116513.0
et: 2757807.0
gn: 1871185.0
bf: 1031330.0
am: 1093499.0
fr: 2110694.0
ie: 1024027.0
dk: 1089958.0
id: 2230820.9
cg: 1115773.0
mz: 1191613.0
us: 2610227.777777778
ro: 1877155.0
sl: 1190761.0
uz: 1978078.0
vn: 2110060.0
pl: 1651676.0
za: 2378977.8
ni: 1140499.0
pk: 3452432.0
sg: 3547809.0
tw: 1656010.0
zm: 1267458.0
mg: 1391506.0
sa: 2404892.3333333335
tr: 3258055.66

In [ ]:
from pyspark.sql.functions import avg, col
from pyspark.sql.window import Window

# Define the threshold population
threshold_population = 1000000

# Calculate the average population for each country using a window function
window = Window.partitionBy("Country")
average_population = avg(col("Population")).over(window)

# Use a subquery to filter the cities with a population above the average population of their respective countries
subquery = dfNew.select("*", average_population.alias("avg_population")).where(col("Population") > col("avg_population"))

# Filter the results in the outer query to include only cities with a population above the specified threshold
filtered_cities = subquery.filter(col("Population") > threshold_population)

# Display the results
print(filtered_cities.count())

filtered_cities.show()


"""
for row in filtered_cities.collect():
    print("{} ,{}, {}".format(row["Country"], row["City"], row["Population"]))
"""

277
+-------+--------------+--------------+------+----------+----------+----------+-------------+------------------+
|Country|          City|    AccentCity|Region|Population|  Latitude| Longitude|rounded_value|    avg_population|
+-------+--------------+--------------+------+----------+----------+----------+-------------+------------------+
|     ae|         dubai|         Dubai|    03|   1137376| 25.258172| 55.304717|            6| 761668.3333333334|
|     ao|        luanda|        Luanda|    20|   2776125| -8.836804| 13.233174|            6|      134849.40625|
|     au|      adelaide|      Adelaide|    05|   1074168|-34.928661|138.598633|            6| 94962.11229946524|
|     au|      brisbane|      Brisbane|    04|   1843459| -27.47101|153.024292|            6| 94962.11229946524|
|     au|     melbourne|     Melbourne|    07|   3730212|-37.813938|144.963425|            6| 94962.11229946524|
|     au|         perth|         Perth|    08|   1446715| -31.95224|115.861397|            6

'\nfor row in filtered_cities.collect():\n    print("{} ,{}, {}".format(row["Country"], row["City"], row["Population"]))\n'

In [ ]:
from pyspark.sql.functions import desc

dfNoDuplicate = dfNew.dropDuplicates(["Country", "Region", "Population"])

sortedDf = dfNoDuplicate.orderBy(desc("Population"))

sortedDf.show()

print(sortedDf.count())
print(dfNew.count())

+-------+---------+----------+------+----------+----------+-----------+-------------+
|Country|     City|AccentCity|Region|Population|  Latitude|  Longitude|rounded_value|
+-------+---------+----------+------+----------+----------+-----------+-------------+
|     jp|    tokyo|     Tokyo|    40|  31480498|    35.685| 139.751389|            7|
|     cn| shanghai|  Shanghai|    23|  14608512| 31.045556| 121.399722|            7|
|     in|   bombay|    Bombay|    16|  12692717|    18.975|  72.825833|            7|
|     pk|  karachi|   Karachi|    05|  11627378|   24.9056|    67.0822|            7|
|     in|    delhi|     Delhi|    07|  10928270| 28.666667|  77.216667|            7|
|     ph|   manila|    Manila|    D9|  10443877|   14.6042|   120.9822|            7|
|     ru|   moscow|    Moscow|    48|  10381288| 55.752222|  37.615556|            7|
|     kr|    seoul|     Seoul|    11|  10323448|   37.5985|   126.9783|            7|
|     br|sao paulo| São Paulo|    27|  10021437|-23.47